# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f25406d7820>
├── 'a' --> tensor([[-0.2222,  1.0879, -0.5440],
│                   [-0.4766, -1.4393, -0.2423]])
└── 'x' --> <FastTreeValue 0x7f25406d7c40>
    └── 'c' --> tensor([[-1.0354,  0.5283,  0.9906,  1.1054],
                        [ 1.4309,  1.2474, -0.3648, -0.1456],
                        [ 1.0511,  0.2885, -1.1168, -1.7835]])

In [4]:
t.a

tensor([[-0.2222,  1.0879, -0.5440],
        [-0.4766, -1.4393, -0.2423]])

In [5]:
%timeit t.a

66.1 ns ± 0.0965 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f25406d7820>
├── 'a' --> tensor([[-0.1789,  0.9562,  0.4780],
│                   [ 0.9567,  1.4572, -1.9092]])
└── 'x' --> <FastTreeValue 0x7f25406d7c40>
    └── 'c' --> tensor([[-1.0354,  0.5283,  0.9906,  1.1054],
                        [ 1.4309,  1.2474, -0.3648, -0.1456],
                        [ 1.0511,  0.2885, -1.1168, -1.7835]])

In [7]:
%timeit t.a = new_value

68.7 ns ± 0.0454 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.2222,  1.0879, -0.5440],
               [-0.4766, -1.4393, -0.2423]]),
    x: Batch(
           c: tensor([[-1.0354,  0.5283,  0.9906,  1.1054],
                      [ 1.4309,  1.2474, -0.3648, -0.1456],
                      [ 1.0511,  0.2885, -1.1168, -1.7835]]),
       ),
)

In [10]:
b.a

tensor([[-0.2222,  1.0879, -0.5440],
        [-0.4766, -1.4393, -0.2423]])

In [11]:
%timeit b.a

57.6 ns ± 0.113 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.0782, -1.4078, -0.2223],
               [-0.0661,  0.2246,  0.1280]]),
    x: Batch(
           c: tensor([[-1.0354,  0.5283,  0.9906,  1.1054],
                      [ 1.4309,  1.2474, -0.3648, -0.1456],
                      [ 1.0511,  0.2885, -1.1168, -1.7835]]),
       ),
)

In [13]:
%timeit b.a = new_value

488 ns ± 0.106 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

820 ns ± 0.189 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.5 µs ± 35.7 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

143 µs ± 1.38 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

143 µs ± 520 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f2473a284c0>
├── 'a' --> tensor([[[-0.2222,  1.0879, -0.5440],
│                    [-0.4766, -1.4393, -0.2423]],
│           
│                   [[-0.2222,  1.0879, -0.5440],
│                    [-0.4766, -1.4393, -0.2423]],
│           
│                   [[-0.2222,  1.0879, -0.5440],
│                    [-0.4766, -1.4393, -0.2423]],
│           
│                   [[-0.2222,  1.0879, -0.5440],
│                    [-0.4766, -1.4393, -0.2423]],
│           
│                   [[-0.2222,  1.0879, -0.5440],
│                    [-0.4766, -1.4393, -0.2423]],
│           
│                   [[-0.2222,  1.0879, -0.5440],
│                    [-0.4766, -1.4393, -0.2423]],
│           
│                   [[-0.2222,  1.0879, -0.5440],
│                    [-0.4766, -1.4393, -0.2423]],
│           
│                   [[-0.2222,  1.0879, -0.5440],
│                    [-0.4766, -1.4393, -0.2423]]])
└── 'x' --> <FastTreeValue 0x7f2473a280a0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.7 µs ± 46.3 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f2473a287c0>
├── 'a' --> tensor([[-0.2222,  1.0879, -0.5440],
│                   [-0.4766, -1.4393, -0.2423],
│                   [-0.2222,  1.0879, -0.5440],
│                   [-0.4766, -1.4393, -0.2423],
│                   [-0.2222,  1.0879, -0.5440],
│                   [-0.4766, -1.4393, -0.2423],
│                   [-0.2222,  1.0879, -0.5440],
│                   [-0.4766, -1.4393, -0.2423],
│                   [-0.2222,  1.0879, -0.5440],
│                   [-0.4766, -1.4393, -0.2423],
│                   [-0.2222,  1.0879, -0.5440],
│                   [-0.4766, -1.4393, -0.2423],
│                   [-0.2222,  1.0879, -0.5440],
│                   [-0.4766, -1.4393, -0.2423],
│                   [-0.2222,  1.0879, -0.5440],
│                   [-0.4766, -1.4393, -0.2423]])
└── 'x' --> <FastTreeValue 0x7f2473a28100>
    └── 'c' --> tensor([[-1.0354,  0.5283,  0.9906,  1.1054],
                        [ 1.4309,  1.2474, -0.3648, -0.1456],
                 

In [23]:
%timeit t_cat(trees)

30.2 µs ± 69.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

66.9 µs ± 96.4 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-1.0354,  0.5283,  0.9906,  1.1054],
                       [ 1.4309,  1.2474, -0.3648, -0.1456],
                       [ 1.0511,  0.2885, -1.1168, -1.7835]],
              
                      [[-1.0354,  0.5283,  0.9906,  1.1054],
                       [ 1.4309,  1.2474, -0.3648, -0.1456],
                       [ 1.0511,  0.2885, -1.1168, -1.7835]],
              
                      [[-1.0354,  0.5283,  0.9906,  1.1054],
                       [ 1.4309,  1.2474, -0.3648, -0.1456],
                       [ 1.0511,  0.2885, -1.1168, -1.7835]],
              
                      [[-1.0354,  0.5283,  0.9906,  1.1054],
                       [ 1.4309,  1.2474, -0.3648, -0.1456],
                       [ 1.0511,  0.2885, -1.1168, -1.7835]],
              
                      [[-1.0354,  0.5283,  0.9906,  1.1054],
                       [ 1.4309,  1.2474, -0.3648, -0.1456],
                       [ 1.0511,  0.2885, -1.1168, -1.7835]],

In [26]:
%timeit Batch.stack(batches)

78.6 µs ± 205 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-1.0354,  0.5283,  0.9906,  1.1054],
                      [ 1.4309,  1.2474, -0.3648, -0.1456],
                      [ 1.0511,  0.2885, -1.1168, -1.7835],
                      [-1.0354,  0.5283,  0.9906,  1.1054],
                      [ 1.4309,  1.2474, -0.3648, -0.1456],
                      [ 1.0511,  0.2885, -1.1168, -1.7835],
                      [-1.0354,  0.5283,  0.9906,  1.1054],
                      [ 1.4309,  1.2474, -0.3648, -0.1456],
                      [ 1.0511,  0.2885, -1.1168, -1.7835],
                      [-1.0354,  0.5283,  0.9906,  1.1054],
                      [ 1.4309,  1.2474, -0.3648, -0.1456],
                      [ 1.0511,  0.2885, -1.1168, -1.7835],
                      [-1.0354,  0.5283,  0.9906,  1.1054],
                      [ 1.4309,  1.2474, -0.3648, -0.1456],
                      [ 1.0511,  0.2885, -1.1168, -1.7835],
                      [-1.0354,  0.5283,  0.9906,  1.1054],
                   

In [28]:
%timeit Batch.cat(batches)

149 µs ± 2.48 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

350 µs ± 3.44 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
